In [1]:
import sys
sys.path.append('..')
import torch
from torchmetrics.classification import BinaryF1Score, F1Score
import wandb
from tqdm import trange
import os
from datetime import datetime
from sklearn import metrics
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import pytorch_lightning as pl
import numpy as np

sys.path.insert(0, '/Users/evanpan/Documents/GitHub/EvansToolBox/Utils')
sys.path.insert(0, '/Users/evanpan/Desktop/openpose/python/')
sys.path.insert(0, '/scratch/ondemand27/evanpan/EvansToolBox/Utils/')
sys.path.insert(0, '/scratch/ondemand27/evanpan/Gaze_project/')

# from training.model import *
from Dataset_Util.dataloader import *

In [2]:
%load_ext autoreload
%autoreload 1

%aimport training.model
%aimport Dataset_Util.dataloader

# Loaded Things

In [3]:
dataset_location = "/scratch/ondemand27/evanpan/data/deep_learning_processed_dataset"
model_save_location = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models"
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/baseline_config.json", "r"))

## Load dataset

In [4]:
# do the training test split here:
dataset_metadata = "/scratch/ondemand27/evanpan/data/deep_learning_processed_dataset/video_to_window_metadata.json"
dataset_metadata = json.load(open(dataset_metadata, "r"))
all_videos = list(dataset_metadata.keys())
training_set = []
testing_set = []
# get the name of the videos (this ensures no contamination because the same shot is split)
for i in range(0, len(all_videos)):
    if i / len(all_videos) < 0.9:
        training_set.append(all_videos[i])
    else:
        testing_set.append(all_videos[i])

In [5]:
class Audio_only_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(Audio_only_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * (config["input_layer_out"]) * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)        
        # output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3)

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        
        text_feature_self = mod_audio_self[:, :, :6]
        mod_audio_self = mod_audio_self[:, :, 6:]
        text_feature_other = mod_audio_self[:, :, :6]
        mod_audio_other = mod_audio_other[:, :, 6:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, x2_windowed], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x
    def load_weights(self, pretrained_dict):
    #   not_copy = set(['fc.weight', 'fc.bias'])
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items()}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [6]:
class SentenceBaseline_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SentenceBaseline_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * (config["input_layer_out"] + 6) * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)        
        # output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3)

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        
        text_feature_self = mod_audio_self[:, :, :6]
        mod_audio_self = mod_audio_self[:, :, 6:]
        text_feature_other = mod_audio_self[:, :, :6]
        mod_audio_other = mod_audio_other[:, :, 6:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, text_feature_self, x2_windowed, text_feature_other], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x
    def load_weights(self, pretrained_dict):
    #   not_copy = set(['fc.weight', 'fc.bias'])
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items()}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [7]:
class SimpleBaseline_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SimpleBaseline_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        
        # output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3, nn.Sigmoid())

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, x2_windowed], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x

In [8]:
class SimpleBaselineTransformer_GazePredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SimpleBaselineTransformer_GazePredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)
        self.trasnformer = nn.Transformer(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), batch_first=True,)
        
        # output layers
        self.output_layer_1 = nn.Linear(2 * config["input_layer_out"] * (self.frames_ahead + self.frames_behind + 1), config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3, nn.Sigmoid())

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, x2_windowed], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)
        return x

In [9]:
class SentenceBaseline_Gaze_and_Direction_PredictionModel(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SentenceBaseline_Gaze_and_Direction_PredictionModel, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * (config["input_layer_out"] + 6) * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)        
        # aversion output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3)

        # directional output layers
        self.directional_output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.directional_output_layer_1 = nn.Sequential(self.directional_output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.directional_output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.directional_output_layer_2 = nn.Sequential(self.directional_output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.directional_output_layer_3 = nn.Linear(config["output_layer_2_hidden"], 4)
        self.directional_output_layer_3 = nn.Sequential(self.directional_output_layer_3)

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        
        text_feature_self = mod_audio_self[:, :, :6]
        mod_audio_self = mod_audio_self[:, :, 6:]
        text_feature_other = mod_audio_self[:, :, :6]
        mod_audio_other = mod_audio_other[:, :, 6:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, text_feature_self, x2_windowed, text_feature_other], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)

        x_dir = self.activation(out)
        x_dir = self.directional_output_layer_1(x_dir)
        x_dir = self.directional_output_layer_2(x_dir)
        x_dir = self.directional_output_layer_3(x_dir)
        return x, x_dir
    def load_weights(self, pretrained_dict):
    #   not_copy = set(['fc.weight', 'fc.bias'])
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items()}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [10]:
class SentenceBaseline_Gaze_and_Direction_PredictionModel_only_updown(nn.Module):
    def __init__(self, config):
        torch.set_default_tensor_type(torch.DoubleTensor)
        # initialize model
        super(SentenceBaseline_Gaze_and_Direction_PredictionModel_only_updown, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.activation = nn.Sigmoid()
        self.num_layers = config["num_layers"]
        self.config = config
        # the feature of each speaker are encoded with a separate Linear Layer
        self.input_layer_self = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        self.input_layer_other = nn.Linear(int(config["input_dims"]/2 - 6), config["input_layer_out"])
        
        # the Recurrent Layer will take care of the next step
        self.lstm_hidden_dims = config["lstm_output_feature_size"]
        self.num_lstm_layer = config["lstm_layer_num"]
        self.frames_ahead = config["frames_ahead"]
        self.frames_behind = config["frames_behind"]
        self.lstm = nn.LSTM(2 * (config["input_layer_out"] + 6) * (self.frames_ahead + self.frames_behind + 1), 
                            self.lstm_hidden_dims, 
                            self.num_lstm_layer, 
                            batch_first=True)        
        # aversion output layers
        self.output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.output_layer_1 = nn.Sequential(self.output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.output_layer_2 = nn.Sequential(self.output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.output_layer_3 = nn.Linear(config["output_layer_2_hidden"], config["output_layer_3_hidden"])
        self.output_layer_3 = nn.Sequential(self.output_layer_3)

        # directional output layers
        self.directional_output_layer_1 = nn.Linear(self.lstm_hidden_dims, config["output_layer_1_hidden"])
        self.directional_output_layer_1 = nn.Sequential(self.directional_output_layer_1, self.activation, nn.Dropout(self.config["dropout"]))
        self.directional_output_layer_2 = nn.Linear(config["output_layer_1_hidden"], config["output_layer_2_hidden"])
        self.directional_output_layer_2 = nn.Sequential(self.directional_output_layer_2, self.activation, nn.Dropout(self.config["dropout"]))
        self.directional_output_layer_3 = nn.Linear(config["output_layer_2_hidden"], 3)
        self.directional_output_layer_3 = nn.Sequential(self.directional_output_layer_3)

        # audio_filler = torch.tensor([[[-36.04365338911715,0.0,0.0,0.0,0.0,0.0,-3.432169450445466e-14,0.0,0.0,0.0,9.64028691651994e-15,0.0,0.0,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715,-36.04365338911715]]]).to(self.device)
        # text_filler = torch.ones([1, 1, 772]).to(self.device) * -15
        # text_filler[:, :, -4:] = 0
        # self.filler = torch.concat([audio_filler, text_filler], axis=2)
    def concate_frames(self, input_feature):
        # here I expect the 
        padding_front = torch.zeros([input_feature.shape[0], self.frames_ahead, input_feature.shape[2]]).to(self.device)
        padding_back = torch.zeros([input_feature.shape[0], self.frames_behind, input_feature.shape[2]]).to(self.device)
        padded_input_audio = torch.cat([padding_front, input_feature, padding_back], dim=1)
        window_audio = []
        for i in range(0, input_feature.shape[1]):
            window_count = i + self.frames_ahead
            current_window = padded_input_audio[:, window_count-self.frames_ahead:window_count+self.frames_behind+1]
            s = current_window.shape
            current_window = current_window.view((s[0], s[1] * s[2]))
            current_window = torch.unsqueeze(current_window, 1)
            window_audio.append(current_window)
        rtv = torch.cat(window_audio, dim=1)
        return rtv
    def forward(self, input_feature):
        feature_size = int(input_feature.size()[2] / 2)
        mod_audio_self = input_feature[:, :, :feature_size]
        mod_audio_other = input_feature[:, :, feature_size:]
        
        text_feature_self = mod_audio_self[:, :, :6]
        mod_audio_self = mod_audio_self[:, :, 6:]
        text_feature_other = mod_audio_self[:, :, :6]
        mod_audio_other = mod_audio_other[:, :, 6:]
        x1 = self.activation(self.input_layer_self(mod_audio_self))
        x2 = self.activation(self.input_layer_self(mod_audio_other))
        x1_windowed = self.concate_frames(x1)
        x2_windowed = self.concate_frames(x2)
        x_combined = torch.concat([x1_windowed, text_feature_self, x2_windowed, text_feature_other], axis=2)
        # here I'm assuming that the input_audio is of proper shape
        out, hidden_state = self.lstm(x_combined)
        # bn
        # x = self.bn(out.permute(0, 2, 1)).permute(0, 2, 1)
        x = self.activation(out)
        x = self.output_layer_1(x)
        x = self.output_layer_2(x)
        x = self.output_layer_3(x)

        x_dir = self.activation(out)
        x_dir = self.directional_output_layer_1(x_dir)
        x_dir = self.directional_output_layer_2(x_dir)
        x_dir = self.directional_output_layer_3(x_dir)
        return x, x_dir
    def load_weights(self, pretrained_dict):
    #   not_copy = set(['fc.weight', 'fc.bias'])
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items()}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

# Training Loop - For baseline model

In [11]:
def train_model(model, config, train_data, valid_data, wandb, model_name):
    optimiser = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train() 
    loss_fn = nn.CrossEntropyLoss()
    training_loss = []
    valid_loss = []
    training_f1 = []
    valid_f1 = []
    aversion_vs_start = []
    count = 0
    # f1_score = BinaryF1Score(num_classes=2).to(device)
    f1_score = F1Score(task="multiclass", num_classes=2, average="weighted").to(device)
    for epoch in range(1, config['epochs'] + 1):
        total_train_loss = 0
        total_valid_loss = 0
        total_aversion_predicted = 0
        total_train_f1 = 0
        total_valid_f1 = 0
        train_batch_counter = 0
        valid_batch_counter = 0
        total_prediction_counter = 0
        prediction_mean = 0
        prediction_std = 0
        model.zero_grad()
        for _, (X, Y) in enumerate(train_data):
            train_batch_counter += 1
            X, Y = X.to(device), Y.to(device)
            optimiser.zero_grad()
            if "Transformer" in config["model_type"]:
                all_zero = torch.zeros(Y.shape).to(device)
                pred = model(X, all_zero)
            else:
                pred = model(X)
            loss = loss_fn(pred.transpose(2, 1), Y.long())
            loss.backward()
            optimiser.step()
            total_train_loss += loss.item()
            # binary_pred = torch.round(pred)
            binary_pred = torch.argmax(pred, axis=2, keepdim=True)
            prediction_mean = torch.mean(binary_pred.float()).item()
            prediction_std = torch.std(binary_pred.float()).item()            
            f1_train = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            total_aversion_predicted += torch.sum(binary_pred).item()
            total_prediction_counter += binary_pred.size()[0] * binary_pred.size()[1] 
            total_train_f1 += f1_train
            del X, Y, pred
            torch.cuda.empty_cache()

        total_train_f1 /= train_batch_counter
        total_train_loss /= len(train_data)
        total_aversion_predicted /= total_prediction_counter

        for _, (X, Y) in enumerate(valid_data):
            with torch.no_grad():
                valid_batch_counter += 1
                X, Y = X.to(device), Y.to(device)
                if "Transformer" in config["model_type"]:
                    all_zero = torch.zeros(Y.shape).to(device)
                    pred = model(X, all_zero)
                else:
                    pred = model(X)
                loss = loss_fn(pred.transpose(2, 1), Y.long())
                total_valid_loss += loss.item()

                # binary_pred = torch.round(pred)
                binary_pred = torch.argmax(pred, axis=2, keepdim=True)
                f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
                total_valid_f1 += f1_valid
                del X, Y, pred
                torch.cuda.empty_cache()

        total_valid_f1 /= valid_batch_counter
        total_valid_loss /= len(valid_data)

        if config['wandb']:
            wandb.log({'training loss': total_train_loss,
                        'validation_loss': total_valid_loss,
                        'training_f1': total_train_f1,
                        'validation_f1': total_valid_f1, 
                        "percentage_predicted_aversion": total_aversion_predicted})
        training_loss.append(total_train_loss)
        valid_loss.append(total_valid_loss)
        training_f1.append(total_train_f1)
        valid_f1.append(total_valid_f1)
        aversion_vs_start.append(total_aversion_predicted)
        if total_valid_f1 == max(valid_f1):
            try:
                os.mkdir(os.path.join(*[model_save_location, model_name]))
            except:
                pass
            config_save_path = os.path.join(*[model_save_location, model_name, "config.json"])
            json.dump(config, open(config_save_path, "w"))
            file_name = f'time={datetime.now()}_epoch={epoch}.pt'
            save_path = os.path.join(*[model_save_location, model_name, file_name])
            torch.save(model.state_dict(), save_path)
        if config['early_stopping']>0:
            if epoch > 1:
                if total_valid_f1 < np.mean(valid_f1[epoch - 7:epoch - 2]):
                    count += 1
                else:
                    count = 0
            if count >= config['early_stopping']:
                print('\n\nStopping early due to decrease in performance on validation set\n\n')
                break 
        if count == 0:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1))
        else:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}, model have not improved for {} iterations".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1, count))
    if config['wandb']:
        save_path = os.path.join(*[model_save_location, model_name, file_name])
        wandb.save(save_path)

## Weight and Biases Stuff

## Train the model

In [57]:
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/baseline_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set[:10])
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set[:2])
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SimpleBaseline_GazePredictionModel(config)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)
run_obj.finish()

  0%|          | 0/200 [00:00<?, ?it/s]

0.03449014124949654


  0%|          | 1/200 [00:02<07:47,  2.35s/it]

mean: 0.5974612089529724, std: 0.03449014124949654
training L: 0.7498041014467824
validation L:0.7801158889905458
0.0387789817215484


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|          | 2/200 [00:04<06:59,  2.12s/it]

...

mean: 0.6587029522108311, std: 0.014475230401421608
training L: 0.7506199214130393
validation L:0.7802088892277198
0.008366695740317093


  3%|▎         | 6/200 [00:12<06:28,  2.00s/it]

mean: 0.6412970440137612, std: 0.008366695740317093
training L: 0.7506199214130393
validation L:0.7802088892277198


  3%|▎         | 6/200 [00:12<06:37,  2.05s/it]


KeyboardInterrupt: 

# Training Loop - For Audio Only model

In [ ]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))

In [20]:
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/audio_only_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set[:12], audio_only=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set[:2], audio_only=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SimpleBaseline_GazePredictionModel(config)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)

  0%|          | 1/200 [00:01<05:35,  1.69s/it]

mean: 0.5375652030339769, std: 0.038422848050369815
training L: 0.7323386721605636
validation L:0.7280215162212137


  1%|          | 2/200 [00:03<05:08,  1.56s/it]

mean: 0.5396894879007264, std: 0.03841229046891618
training L: 0.7376692695463082
validation L:0.7348674337168585


  2%|▏         | 3/200 [00:04<04:59,  1.52s/it]

...

100%|█████████▉| 199/200 [04:31<00:01,  1.37s/it]

mean: 0.652308709116172, std: 0.038421576275752184
training L: 0.7891518531254205
validation L:0.7801753717117804


100%|██████████| 200/200 [04:32<00:00,  1.36s/it]
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


mean: 0.6524606512561358, std: 0.038428353510256905
training L: 0.7890322484189752
validation L:0.7802348635046515


'time=2023-04-04 20:54:46.715070_epoch=196.pt'

# Training Loop - For sentence level feature Only model (This is actually for word level only)

In [14]:
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/word_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set, word_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, word_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
model.to(device)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)

NameError: name 'A' is not defined

## further train the model

In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/word_config.json", "r"))
model = SentenceBaseline_GazePredictionModel(config)
config["learning_rate"] = 0.00001
config["load_model"] = True
if config["wandb"]:
    wandb.login()
    if config["load_model"]:
        run_obj = wandb.init(project="gaze_prediction", config=config, save_code=True,
            resume='allow', id='8w9fyxan')
        # checkpoint_name = "gaze_prediction_team/gaze_prediction/8w9fyxan"
        checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/time=2023-04-05 02:37:34.205141_epoch=200.pt"
        wandb.restore(checkpoint_path)
        pretrained_dict = torch.load(checkpoint_path, map_location=device)
        model.load_weights(pretrained_dict)
        model.to(config['device'])
    else:
        run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
else:
    run_obj = None

training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)

NameError: name 'torch' is not defined

# Training Loop - For sentence level feature Only model (It's real this time)

In [11]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
model.to(device)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, "sentence_and_words")
run_obj.finish()


Epoch 1, mean: 0.17789314687252045, std: 0.3824285864830017
training L: 0.38361384790041525
validation L:0.5202852326801098


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 2, mean: 0.5589313507080078, std: 0.49652254581451416
training L: 0.5135592779060695
validation L:0.5422346910798534
Epoch 3, mean: 0.8715725541114807, std: 0.3345702886581421
training L: 0.5083247797239047
validation L:0.5124457632512047, model have not improved for 1 iterations
Epoch 4, mean: 0.9777710437774658, std: 0.14742977917194366
training L: 0.4620488325757937
validation L:0.5003760834120156, model have not improved for 2 iterations
Epoch 5, mean: 0.9966717958450317, std: 0.05759573355317116
training L: 0.45366407573439577
validation L:0.49920991627236866
Epoch 6, mean: 0.9993893504142761, std: 0.024704914540052414
training L: 0.45135263240753554
validation L:0.49904111077228025
Epoch 7, mean: 0.999908447265625, std: 0.009570656344294548
training L: 0.4510088849744354
validation L:0.4989816254955779, model have not improved for 1 iterations
Epoch 8, mean: 0.9999695420265198, std: 0.005525789689272642
training L: 0.4510375310310284
validation L:0.49904111077228025, model 

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")




Stopping early due to decrease in performance on validation set




percentage_predicted_aversion,▁▄▇██████████████████████████████
training loss,█▅▃▂▂▂▂▂▃▃▃▂▂▂▂▂▁▁▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁
training_f1,▁██▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
validation_f1,▄█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁
validation_loss,█▅▃▂▂▁▂▂▂▁▂▁▁▂▂▂▂▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁
percentage_predicted_aversion,0.99969
training loss,0.67176
training_f1,0.45141
validation_f1,0.49892
validation_loss,0.65346


# Training Loop for Audio only model

In [8]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = Audio_only_GazePredictionModel(config)
model.to(device)
train_model(model, config, train_dataloader, valid_dataloader, run_obj, "auduo_only")
run_obj.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evanpan (gaze_prediction_team). Use `wandb login --relogin` to force relogin
/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



Epoch 1, mean: 0.9997344613075256, std: 0.01629398763179779
training L: 0.47890092625845143
validation L:0.45902620869931754


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 2, mean: 0.9985726475715637, std: 0.03775409981608391
training L: 0.4761081263856524
validation L:0.49461128031147594
Epoch 3, mean: 0.9990373849868774, std: 0.03101200982928276
training L: 0.47342587689646776
validation L:0.5123245934337597
Epoch 4, mean: 0.9995021224021912, std: 0.022308869287371635
training L: 0.47450191397238706
validation L:0.505301360657689
Epoch 5, mean: 0.9994522929191589, std: 0.023397155106067657
training L: 0.4732315941850137
validation L:0.4702578544176642
Epoch 6, mean: 0.9996514916419983, std: 0.018666332587599754
training L: 0.4739805054340718
validation L:0.4757426025390734
Epoch 7, mean: 0.9998008608818054, std: 0.014111476019024849
training L: 0.47308891153647037
validation L:0.5143512821763536
Epoch 8, mean: 0.9998008608818054, std: 0.014111476019024849
training L: 0.47313888515685837
validation L:0.48575649921554076, model have not improved for 1 iterations
Epoch 9, mean: 0.999817430973053, std: 0.01351082231849432
training L: 0.47322355316679

KeyboardInterrupt: 

# Training for sentence + word + audio. but also uses velocity as a loss function

In [12]:
def get_velocity(out):
    vel = torch.diff(out, dim=1, prepend=out[:, 0:1, :])
    return vel
def train_model_with_vel(model, config, train_data, valid_data, wandb, model_name, start = 1):
    optimiser = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train() 
    loss_fn = nn.CrossEntropyLoss()
    loss_fn_vel = nn.MSELoss()
    training_loss = []
    valid_loss = []
    training_f1 = []
    valid_f1 = []
    aversion_vs_start = []
    count = 0
    # f1_score = BinaryF1Score(num_classes=2).to(device)
    f1_score = F1Score(task="multiclass", num_classes=2, average="weighted").to(device)
    for epoch in range(start, config['epochs'] + 1):
        total_train_loss = 0
        total_valid_loss = 0
        total_aversion_predicted = 0
        total_train_f1 = 0
        total_valid_f1 = 0
        train_batch_counter = 0
        valid_batch_counter = 0
        total_prediction_counter = 0
        prediction_mean = 0
        prediction_std = 0
        model.zero_grad()
        for _, (X, [Y, Y_vel]) in enumerate(train_data):
            train_batch_counter += 1
            X, Y, Y_vel = X.to(device), Y.to(device), Y_vel.to(device)            
            optimiser.zero_grad()
            if "Transformer" in config["model_type"]:
                all_zero = torch.zeros(Y.shape).to(device)
                pred = model(X, all_zero)
            else:
                pred = model(X)
            loss = loss_fn(pred.transpose(2, 1), Y.long())
            # get the softmax
            pred_vel = get_velocity(torch.softmax(pred, axis=2))
            # add the velocity along dimension 1 and 2 respectively, the velocity are opposite 
            loss = loss + 0.3 * loss_fn_vel(pred_vel[:, :, 1], Y_vel)
            loss = loss + 0.3 * loss_fn_vel(pred_vel[:, :, 0], -Y_vel)
            loss.backward()
            optimiser.step()
            total_train_loss += loss.item()
            # binary_pred = torch.round(pred)
            binary_pred = torch.argmax(pred, axis=2, keepdim=True)
            prediction_mean = torch.mean(binary_pred.float()).item()
            prediction_std = torch.std(binary_pred.float()).item()            
            f1_train = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            total_aversion_predicted += torch.sum(binary_pred).item()
            total_prediction_counter += binary_pred.size()[0] * binary_pred.size()[1] 
            total_train_f1 += f1_train
            del X, Y, pred
            torch.cuda.empty_cache()

        total_train_f1 /= train_batch_counter
        total_train_loss /= len(train_data)
        total_aversion_predicted /= total_prediction_counter

        for _, (X, [Y, Y_vel]) in enumerate(valid_data):
            with torch.no_grad():
                valid_batch_counter += 1
                X, Y = X.to(device), Y.to(device)
                if "Transformer" in config["model_type"]:
                    all_zero = torch.zeros(Y.shape).to(device)
                    pred = model(X, all_zero)
                else:
                    pred = model(X)
                loss = loss_fn(pred.transpose(2, 1), Y.long())
                total_valid_loss += loss.item()

                # binary_pred = torch.round(pred)
                binary_pred = torch.argmax(pred, axis=2, keepdim=True)
                f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
                total_valid_f1 += f1_valid
                del X, Y, pred
                torch.cuda.empty_cache()

        total_valid_f1 /= valid_batch_counter
        total_valid_loss /= len(valid_data)

        if config['wandb']:
            wandb.log({'training loss': total_train_loss,
                        'validation_loss': total_valid_loss,
                        'training_f1': total_train_f1,
                        'validation_f1': total_valid_f1, 
                        "percentage_predicted_aversion": total_aversion_predicted})
        training_loss.append(total_train_loss)
        valid_loss.append(total_valid_loss)
        training_f1.append(total_train_f1)
        valid_f1.append(total_valid_f1)
        aversion_vs_start.append(total_aversion_predicted)
        if total_valid_f1 == max(valid_f1):
            try:
                os.mkdir(os.path.join(*[model_save_location, model_name]))
            except:
                pass
            config_save_path = os.path.join(*[model_save_location, model_name, "config.json"])
            json.dump(config, open(config_save_path, "w"))
            file_name = f'time={datetime.now()}_epoch={epoch}.pt'
            save_path = os.path.join(*[model_save_location, model_name, file_name])
            torch.save(model.state_dict(), save_path)
        if config['early_stopping']>0:
            if epoch > 1:
                if total_valid_f1 < np.mean(valid_f1[epoch - 7:epoch - 2]):
                    count += 1
                else:
                    count = 0
            if count >= config['early_stopping']:
                print('\n\nStopping early due to decrease in performance on validation set\n\n')
                break 
        if count == 0:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1))
        else:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}, model have not improved for {} iterations".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1, count))
    if config['wandb']:
        save_path = os.path.join(*[model_save_location, model_name, file_name])
        wandb.save(save_path)

In [12]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
model.to(device)
training_dataset[0]
train_model_with_vel(model, config, train_dataloader, valid_dataloader, run_obj, "sentence_and_words_with_vel")
run_obj.finish()


wandb: ERROR Control-C detected -- Run data was not synced


NameError: name 'train_model_with_vel' is not defined

### Continue training

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
model = SentenceBaseline_GazePredictionModel(config)
config["learning_rate"] = 0.001
config["load_model"] = True
if config["wandb"]:
    wandb.login()
    if config["load_model"]:
        run_obj = wandb.init(project="gaze_prediction", config=config, save_code=True,
            resume='allow', id='0x1rrp9p')
        # checkpoint_name = "gaze_prediction_team/gaze_prediction/8w9fyxan"
        checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/sentence_and_words_with_vel/time=2023-04-10 17:21:49.696880_epoch=267.pt"
        wandb.restore(checkpoint_path)
        pretrained_dict = torch.load(checkpoint_path, map_location=device)
        model.load_weights(pretrained_dict)
        model.to(device)
    else:
        run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
else:
    run_obj = None

training_dataset = Aversion_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)

train_model_with_vel(model, config, train_dataloader, valid_dataloader, run_obj, "sentence_and_words_with_vel", 289)
run_obj.finish()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


percentage_predicted_aversion,0.63243
training loss,0.16717
training_f1,0.92708
validation_f1,0.91199
validation_loss,0.18008


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 289, mean: 0.727568507194519, std: 0.4452143907546997
training L: 0.8862669987488125
validation L:0.8699159448302187
Epoch 290, mean: 0.6913692951202393, std: 0.46193215250968933
training L: 0.9023071043049081
validation L:0.9032333195554947
Epoch 291, mean: 0.6280165910720825, std: 0.48333799839019775
training L: 0.9212732141318789
validation L:0.9204383766673967
Epoch 292, mean: 0.6386224031448364, std: 0.4804036617279053
training L: 0.9278142231326103
validation L:0.9182339477268722
Epoch 293, mean: 0.6577759385108948, std: 0.47445806860923767
training L: 0.9279337160812661
validation L:0.919067533646487
Epoch 294, mean: 0.6244979500770569, std: 0.4842562973499298
training L: 0.9294962620281899
validation L:0.9166458217475921
Epoch 295, mean: 0.636564314365387, std: 0.48099276423454285
training L: 0.9304094280362858
validation L:0.9090100685655462
Epoch 296, mean: 0.6215767860412598, std: 0.4849979281425476
training L: 0.9275395727425941
validation L:0.8954359215854675
Epoch 2

# Training for sentence + word + audio. but also uses velocity as a loss function, also predict gaze Direction

In [13]:
def get_velocity(out):
    vel = torch.diff(out, dim=1, prepend=out[:, 0:1, :])
    return vel
def train_model_with_for_direction(model, config, train_data, valid_data, wandb, model_name, start = 1, num_of_classes=4):
    optimiser = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train() 
    loss_fn = nn.CrossEntropyLoss()
    loss_fn2 = nn.CrossEntropyLoss()
    loss_fn_vel = nn.MSELoss()
    training_loss = []
    valid_loss = []
    training_f1 = []
    valid_f1 = []
    aversion_vs_start = []
    training_direction_pred_f1 = []
    valid_direction_pred_f1 = []
    count = 0
    # f1_score = BinaryF1Score(num_classes=2).to(device)
    f1_score = F1Score(task="multiclass", num_classes=2, average="weighted").to(device)
    f1_score_direction = F1Score(task="multiclass", num_classes=num_of_classes, average="weighted").to(device)
    for epoch in range(start, start + config['epochs'] + 1):
        total_train_loss = 0
        total_valid_loss = 0
        total_aversion_predicted = 0
        total_train_f1 = 0
        total_valid_f1 = 0
        total_train_direction_f1 = 0
        total_valid_direction_f1 = 0
        train_batch_counter = 0
        valid_batch_counter = 0
        total_prediction_counter = 0
        prediction_mean = 0
        prediction_std = 0
        model.zero_grad()
        for _, (X, [Y, Y_vel, Y_dir]) in enumerate(train_data):
            train_batch_counter += 1
            X, Y, Y_vel, Y_dir= X.to(device), Y.to(device), Y_vel.to(device), Y_dir.to(device)   
            optimiser.zero_grad()
            if "Transformer" in config["model_type"]:
                all_zero = torch.zeros(Y.shape).to(device)
                pred, dire_pred = model(X, all_zero)
            else:
                pred, dire_pred = model(X)
            loss = loss_fn(pred.transpose(2, 1), Y.long())
            # loss for the directional classification
            if epoch <= 100:
                factor = 0.1
            else:
                factor = 1
            loss = loss + factor * loss_fn2(dire_pred.transpose(2, 1), torch.argmax(Y_dir, axis=2).long())
            # get the softmax
            pred_vel = get_velocity(torch.softmax(pred, axis=2))
            # add the velocity along dimension 1 and 2 respectively, the velocity are opposite 
            loss = loss + 0.3 * loss_fn_vel(pred_vel[:, :, 1], Y_vel)
            loss.backward()
            optimiser.step()
            total_train_loss += loss.item()
            # binary_pred = torch.round(pred)
            binary_pred = torch.argmax(pred, axis=2, keepdim=True)
            dire_pred = torch.argmax(dire_pred, axis=2, keepdim=True)
            f1_train = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            f1_dire_train = f1_score_direction(dire_pred, torch.argmax(Y_dir, axis=2, keepdim=True)).item()
            
            prediction_mean = torch.mean(binary_pred.float()).item()
            prediction_std = torch.std(binary_pred.float()).item()            
            
            total_aversion_predicted += torch.sum(binary_pred).item()
            total_prediction_counter += binary_pred.size()[0] * binary_pred.size()[1] 
            total_train_f1 += f1_train
            total_train_direction_f1 += f1_dire_train
            del X, Y, pred, Y_vel, Y_dir, dire_pred
            torch.cuda.empty_cache()

        total_train_f1 /= train_batch_counter
        total_train_direction_f1 /= train_batch_counter
        total_train_loss /= len(train_data)
        total_aversion_predicted /= total_prediction_counter
        
        for _, (X, [Y, Y_vel, Y_dir]) in enumerate(valid_data):
            with torch.no_grad():
                valid_batch_counter += 1
                X, Y, Y_vel, Y_dir= X.to(device), Y.to(device), Y_vel.to(device), Y_dir.to(device)   
                if "Transformer" in config["model_type"]:
                    all_zero = torch.zeros(Y.shape).to(device)
                    pred, dire_pred = model(X, all_zero)
                else:
                    pred, dire_pred = model(X)
                loss = loss_fn(pred.transpose(2, 1), Y.long())
                loss = loss + loss_fn2(dire_pred.transpose(2, 1), torch.argmax(Y_dir, axis=2).long())
                total_valid_loss += loss.item()

                # binary_pred = torch.round(pred)
                binary_pred = torch.argmax(pred, axis=2, keepdim=True)
                f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            
                dire_pred = torch.argmax(dire_pred, axis=2, keepdim=True)
                f1_dire_valid = f1_score_direction(dire_pred, torch.argmax(Y_dir, axis=2, keepdim=True)).item()

                total_valid_f1 += f1_valid
                total_valid_direction_f1 += f1_dire_valid
                del X, Y, pred
                torch.cuda.empty_cache()

        total_valid_f1 /= valid_batch_counter
        total_valid_direction_f1 /= valid_batch_counter
        total_valid_loss /= len(valid_data)

        if config['wandb']:
            wandb.log({'training loss': total_train_loss,
                        'validation_loss': total_valid_loss,
                        'training_f1': total_train_f1,
                        'validation_f1': total_valid_f1, 
                        'training_f1_direction': total_train_direction_f1,
                        'validation_f1_direction': total_valid_direction_f1,
                        "percentage_predicted_aversion": total_aversion_predicted})
        training_loss.append(total_train_loss)
        valid_loss.append(total_valid_loss)
        training_f1.append(total_train_f1)
        valid_f1.append(total_valid_f1)
        aversion_vs_start.append(total_aversion_predicted)
        if total_valid_f1 == max(valid_f1):
            try:
                os.mkdir(os.path.join(*[model_save_location, model_name]))
            except:
                pass
            config_save_path = os.path.join(*[model_save_location, model_name, "config.json"])
            json.dump(config, open(config_save_path, "w"))
            file_name = f'time={datetime.now()}_epoch={epoch}.pt'
            save_path = os.path.join(*[model_save_location, model_name, file_name])
            torch.save(model.state_dict(), save_path)
        if config['early_stopping']>0:
            if epoch > 1:
                if total_valid_f1 < np.mean(valid_f1[epoch - 7:epoch - 2]):
                    count += 1
                else:
                    count = 0
            if count >= config['early_stopping']:
                print('\n\nStopping early due to decrease in performance on validation set\n\n')
                break 
        if count == 0:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1))
        else:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}, model have not improved for {} iterations".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1, count))
    if config['wandb']:
        save_path = os.path.join(*[model_save_location, model_name, file_name])
        wandb.save(save_path)

In [18]:
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config2.json", "r"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_and_Directions_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True)
validation_dataset = Aversion_and_Directions_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_Gaze_and_Direction_PredictionModel(config)
model.to(device)
# training_dataset[0]
train_model_with_for_direction(model, config, train_dataloader, valid_dataloader, run_obj, "aversion_and_direction_larg_batch")
run_obj.finish()

Problem at: /tmp/ipykernel_64833/1140664169.py 1 <module>


KeyboardInterrupt: 

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/data/Gaze_aversion_models/aversion_and_direction_larg_batch/config.json", "r"))
model = SentenceBaseline_Gaze_and_Direction_PredictionModel(config)
config["load_model"] = True
if config["wandb"]:
    wandb.login()
    if config["load_model"]:
        run_obj = wandb.init(project="gaze_prediction", config=config, save_code=True,
            resume='allow', id='vgl6tnx8')
        # checkpoint_name = "gaze_prediction_team/gaze_prediction/8w9fyxan"
        checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/aversion_and_direction_larg_batch/time=2023-04-18 09:01:11.503968_epoch=1986.pt"
        wandb.restore(checkpoint_path)
        pretrained_dict = torch.load(checkpoint_path, map_location=device)
        model.load_weights(pretrained_dict)
        model.to(device)
    else:
        run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
else:
    run_obj = None

training_dataset = Aversion_and_Directions_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True)
validation_dataset = Aversion_and_Directions_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)

train_model_with_for_direction(model, config, train_dataloader, valid_dataloader, run_obj, "aversion_and_direction_larg_batch", 2000)
run_obj.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evanpan (gaze_prediction_team). Use `wandb login --relogin` to force relogin


/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/scratch/ondemand27/evanpan/conda_env/.conda/envs/jaligaze/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 2000, mean: 0.5964974761009216, std: 0.49060043692588806
training L: 0.7636961073059446
validation L:0.7689137324044213
Epoch 2001, mean: 0.7000855207443237, std: 0.4582207500934601
training L: 0.76545473908605
validation L:0.7659171881928002
Epoch 2002, mean: 0.6969318985939026, std: 0.45958492159843445
training L: 0.7615947322489782
validation L:0.7623851474615961
Epoch 2003, mean: 0.6235925555229187, std: 0.4844846725463867
training L: 0.7680084032317864
validation L:0.762308164834449
Epoch 2004, mean: 0.6037479043006897, std: 0.4891184866428375
training L: 0.7716686227492106
validation L:0.7716569796785826
Epoch 2005, mean: 0.6100979447364807, std: 0.48772838711738586
training L: 0.7708461095801616
validation L:0.7730724027016194
Epoch 2006, mean: 0.6527360677719116, std: 0.4761010408401489
training L: 0.7702094992859493
validation L:0.7732597568345345
Epoch 2007, mean: 0.6747980117797852, std: 0.4684508144855499
training L: 0.7696129638568975
validation L:0.7695217552467748


wandb: Network error (ReadTimeout), entering retry loop.


Epoch 2013, mean: 0.6394800543785095, std: 0.480151891708374
training L: 0.7756743898189092
validation L:0.7756401977547731
Epoch 2014, mean: 0.6344783306121826, std: 0.48157668113708496
training L: 0.7753428178748879
validation L:0.7770119938721615
Epoch 2015, mean: 0.6487113237380981, std: 0.47737351059913635
training L: 0.7757421687511182
validation L:0.7768912798836576
Epoch 2016, mean: 0.6484209299087524, std: 0.47746387124061584
training L: 0.7760938260563196
validation L:0.7752476759948539
Epoch 2017, mean: 0.6456319689750671, std: 0.47832190990448
training L: 0.7760918199074245
validation L:0.775854221338377
Epoch 2018, mean: 0.6496117115020752, std: 0.47709208726882935
training L: 0.7763709893244655
validation L:0.7750653899924546
Epoch 2019, mean: 0.6668227314949036, std: 0.471349835395813
training L: 0.7767098275815247
validation L:0.776819389594618
Epoch 2020, mean: 0.6682025790214539, std: 0.47085919976234436
training L: 0.7758314223377502
validation L:0.7777598101194084
E

ValueError: Cannot load file containing pickled data when allow_pickle=False

# Training for sentence + word + audio. but also uses velocity as a loss function, also predict gaze Direction but only up and down

In [12]:
run_obj = None
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config2.json", "r"))
run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# config["wandb"]=False
print(device)
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
training_dataset = Aversion_and_Directions_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True, simple_dir=True)
validation_dataset = Aversion_and_Directions_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True, simple_dir=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_Gaze_and_Direction_PredictionModel_only_updown(config)
model.to(device)
# training_dataset[0]
train_model_with_for_direction(model, config, train_dataloader, valid_dataloader, run_obj, "aversion_and_direction_larg_batch_simple_dir", num_of_classes=3)
run_obj.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evanpan (gaze_prediction_team). Use `wandb login --relogin` to force relogin


cuda
Epoch 1, mean: 1.0, std: 0.0
training L: 0.46306652828977884
validation L:0.49493222482845023


KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/data/Gaze_aversion_models/aversion_and_direction_larg_batch_simple_dir/config.json", "r"))
model = SentenceBaseline_Gaze_and_Direction_PredictionModel_only_updown(config)
config["load_model"] = True
if config["wandb"]:
    wandb.login()
    if config["load_model"]:
        run_obj = wandb.init(project="gaze_prediction", config=config, save_code=True,
            resume='allow', id='l9vqiqwb')
        # checkpoint_name = "gaze_prediction_team/gaze_prediction/8w9fyxan"
        checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/aversion_and_direction_larg_batch_simple_dir/time=2023-04-19 08:28:15.969363_epoch=973.pt"
        wandb.restore(checkpoint_path)
        pretrained_dict = torch.load(checkpoint_path, map_location=device)
        model.load_weights(pretrained_dict)
        model.to(device)
    else:
        run_obj = wandb.init(project="gaze_prediction", config=config, settings=wandb.Settings(start_method="fork"))
else:
    run_obj = None

training_dataset = Aversion_and_Directions_SelfTap111(dataset_location, training_set, sentence_and_word_timing=True, velocity_label=True)
validation_dataset = Aversion_and_Directions_SelfTap111(dataset_location, testing_set, sentence_and_word_timing=True, velocity_label=True)
train_dataloader = torch.utils.data.DataLoader(training_dataset, config['batch_size'], True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)

train_model_with_for_direction(model, config, train_dataloader, valid_dataloader, run_obj, "aversion_and_direction_larg_batch", 2000)
run_obj.finish()


# Training Loop for concatenating input so the model is trained on longer sequences

In [ ]:
def train_model_concat(model, config, train_data, valid_data, wandb, model_name):
    optimiser = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train() 
    loss_fn = nn.CrossEntropyLoss()
    training_loss = []
    valid_loss = []
    training_f1 = []
    valid_f1 = []
    aversion_vs_start = []
    count = 0
    # f1_score = BinaryF1Score(num_classes=2).to(device)
    f1_score = F1Score(task="multiclass", num_classes=2, average="weighted").to(device)
    for epoch in range(1, config['epochs'] + 1):
        total_train_loss = 0
        total_valid_loss = 0
        total_aversion_predicted = 0
        total_train_f1 = 0
        total_valid_f1 = 0
        train_batch_counter = 0
        valid_batch_counter = 0
        total_prediction_counter = 0
        prediction_mean = 0
        prediction_std = 0
        model.zero_grad()
        for _, (X, Y) in enumerate(train_data):
            train_batch_counter += 1
            X, Y = X.to(device), Y.to(device)
            optimiser.zero_grad()
            if "Transformer" in config["model_type"]:
                all_zero = torch.zeros(Y.shape).to(device)
                pred = model(X, all_zero)
            else:
                pred = model(X)
            loss = loss_fn(pred.transpose(2, 1), Y.long())
            loss.backward()
            optimiser.step()
            total_train_loss += loss.item()
            # binary_pred = torch.round(pred)
            binary_pred = torch.argmax(pred, axis=2, keepdim=True)
            prediction_mean = torch.mean(binary_pred.float()).item()
            prediction_std = torch.std(binary_pred.float()).item()            
            f1_train = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
            total_aversion_predicted += torch.sum(binary_pred).item()
            total_prediction_counter += binary_pred.size()[0] * binary_pred.size()[1] 
            total_train_f1 += f1_train
            del X, Y, pred
            torch.cuda.empty_cache()

        total_train_f1 /= train_batch_counter
        total_train_loss /= len(train_data)
        total_aversion_predicted /= total_prediction_counter

        for _, (X, Y) in enumerate(valid_data):
            with torch.no_grad():
                valid_batch_counter += 1
                X, Y = X.to(device), Y.to(device)
                if "Transformer" in config["model_type"]:
                    all_zero = torch.zeros(Y.shape).to(device)
                    pred = model(X, all_zero)
                else:
                    pred = model(X)
                loss = loss_fn(pred.transpose(2, 1), Y.long())
                total_valid_loss += loss.item()

                # binary_pred = torch.round(pred)
                binary_pred = torch.argmax(pred, axis=2, keepdim=True)
                f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
                total_valid_f1 += f1_valid
                del X, Y, pred
                torch.cuda.empty_cache()

        total_valid_f1 /= valid_batch_counter
        total_valid_loss /= len(valid_data)

        if config['wandb']:
            wandb.log({'training loss': total_train_loss,
                        'validation_loss': total_valid_loss,
                        'training_f1': total_train_f1,
                        'validation_f1': total_valid_f1, 
                        "percentage_predicted_aversion": total_aversion_predicted})
        training_loss.append(total_train_loss)
        valid_loss.append(total_valid_loss)
        training_f1.append(total_train_f1)
        valid_f1.append(total_valid_f1)
        aversion_vs_start.append(total_aversion_predicted)
        if total_valid_f1 == max(valid_f1):
            try:
                os.mkdir(os.path.join(*[model_save_location, model_name]))
            except:
                pass
            config_save_path = os.path.join(*[model_save_location, model_name, "config.json"])
            json.dump(config, open(config_save_path, "w"))
            file_name = f'time={datetime.now()}_epoch={epoch}.pt'
            save_path = os.path.join(*[model_save_location, model_name, file_name])
            torch.save(model.state_dict(), save_path)
        if config['early_stopping']>0:
            if epoch > 1:
                if total_valid_f1 < np.mean(valid_f1[epoch - 7:epoch - 2]):
                    count += 1
                else:
                    count = 0
            if count >= config['early_stopping']:
                print('\n\nStopping early due to decrease in performance on validation set\n\n')
                break 
        if count == 0:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1))
        else:
            print("Epoch {}, mean: {}, std: {}\ntraining L: {}\nvalidation L:{}, model have not improved for {} iterations".format(epoch, prediction_mean, prediction_std, total_train_f1, total_valid_f1, count))
    if config['wandb']:
        save_path = os.path.join(*[model_save_location, model_name, file_name])
        wandb.save(save_path)

# Look at the output of the model

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = json.load(open("/scratch/ondemand27/evanpan/Gaze_project/training/sentence_config.json", "r"))
# obtain the dataset
torch.set_default_tensor_type(torch.DoubleTensor)
validation_dataset = Aversion_SelfTap111(dataset_location, testing_set, word_timing=True)
valid_dataloader = torch.utils.data.DataLoader(validation_dataset, config['batch_size'], True)
model = SentenceBaseline_GazePredictionModel(config)
checkpoint_path = "/scratch/ondemand27/evanpan/data/Gaze_aversion_models/time=2023-04-05 02:37:34.205141_epoch=200.pt"
pretrained_dict = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(pretrained_dict)
# train_model(model, config, train_dataloader, valid_dataloader, run_obj, model_save_location)


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



<All keys matched successfully>

In [10]:
for _, (X, Y) in enumerate(valid_data):
    with torch.no_grad():
        valid_batch_counter += 1
        X, Y = X.to(device), Y.to(device)
        if "Transformer" in config["model_type"]:
            all_zero = torch.zeros(Y.shape).to(device)
            pred = model(X, all_zero)
        else:
            pred = model(X)
        # binary_pred = torch.round(pred)
        binary_pred = torch.argmax(pred, axis=2, keepdim=True)
        f1_valid = f1_score(binary_pred, torch.unsqueeze(Y, axis=2)).item()
        total_valid_f1 += f1_valid
        del X, Y, pred
        torch.cuda.empty_cache()

NameError: name 'valid_data' is not defined

In [32]:
model.to(device)
i = 11
x = torch.from_numpy(validation_dataset[i][0]).to(device)
y = validation_dataset[i][1]
pred = model(torch.unsqueeze(x, axis=0))
pred = torch.softmax(pred, dim=2)
pred = pred.cpu().detach().numpy()
plt.plot(pred[0, :, 0], label="prediction")
plt.plot(y, label="label")
plt.legend()


IndexError: index 2 is out of bounds for dimension 2 with size 2